In [28]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import pandas as pd
from MatrixVectorizer import *
import torch
import random
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split

In [29]:
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

# Check for CUDA (GPU support) and set device accordingly
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA is available. Using GPU.")
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # For multi-GPU setups
    # Additional settings for ensuring reproducibility on CUDA
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(device)

mps


In [30]:
A_LR_train = pd.read_csv("../data/lr_train.csv")
A_HR_train = pd.read_csv("../data/hr_train.csv")
A_LR_test = pd.read_csv("../data/lr_test.csv")

In [31]:
LR_size = 160
HR_size = 268

In [32]:
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

A_HR_train = pd.read_csv("../data/hr_train.csv")

pca = PCA(n_components=0.99, whiten=False)
A_HR_train_pca = pca.fit_transform(A_HR_train)
print(A_HR_train_pca.shape)

gm = GaussianMixture(n_components=5, random_state=random_seed)
A_HR_train_label = gm.fit_predict(A_HR_train_pca)
unique, counts = np.unique(A_HR_train_label, return_counts=True)
print(np.asarray((unique, counts)).T)

X = np.load('A_LR_train_matrix.npy')
y = np.load('A_HR_train_matrix.npy')

n_sample = X.shape[0]
X_train, X_val, y_train, y_val = train_test_split(
    X.reshape(n_sample, -1), 
    y.reshape(n_sample, -1), 
    test_size=0.20, 
    random_state=random_seed,
    stratify=A_HR_train_label
)

X_train = X_train.reshape(-1, LR_size, LR_size)
X_val = X_val.reshape(-1, LR_size, LR_size)
y_train = y_train.reshape(-1, HR_size, HR_size)
y_val = y_val.reshape(-1, HR_size, HR_size)

print("Train size:", len(X_train))
print("Val size:", len(X_val))

(167, 161)
[[ 0 13]
 [ 1 41]
 [ 2 14]
 [ 3 48]
 [ 4 51]]
Train size: 133
Val size: 34


In [33]:
# def antivectorize_df(adj_mtx_df, size):
    
#     num_subject = adj_mtx_df.shape[0]
#     adj_mtx = np.zeros((num_subject, size, size)) #torch.zeros((num_subject, LR_size, LR_size))
#     for i in range(num_subject):
#         adj_mtx[i] = MatrixVectorizer.anti_vectorize(adj_mtx_df.iloc[i], size) # torch.from_numpy(MatrixVectorizer.anti_vectorize(A_LR_train.iloc[i], LR_size))
#     return adj_mtx

# np.save('A_LR_train_matrix.npy', antivectorize_df(A_LR_train, LR_size))
# np.save('A_HR_train_matrix.npy', antivectorize_df(A_HR_train, HR_size))
# np.save('A_LR_test_matrix.npy', antivectorize_df(A_LR_test, LR_size))

In [34]:
A_LR_train_matrix = np.load('A_LR_train_matrix.npy')
A_HR_train_matrix = np.load('A_HR_train_matrix.npy')
A_LR_test_matrix = np.load("A_LR_test_matrix.npy")

print(A_LR_train_matrix.shape)
print(A_HR_train_matrix.shape)
print(A_LR_test_matrix.shape)

(167, 160, 160)
(167, 268, 268)
(112, 160, 160)


In [35]:
"""Main function of Graph Super-Resolution Network (GSR-Net) framework 
   for predicting high-resolution brain connectomes from low-resolution connectomes. 
    
    ---------------------------------------------------------------------
    
    This file contains the implementation of the training and testing process of our GSR-Net model.
        train(model, optimizer, subjects_adj, subjects_ground_truth, args)

                Inputs:
                        model:        constructor of our GSR-Net model:  model = GSRNet(ks,args)
                                      ks:   array that stores reduction rates of nodes in Graph U-Net pooling layers
                                      args: parsed command line arguments

                        optimizer:    constructor of our model's optimizer (borrowed from PyTorch)  

                        subjects_adj: (n × l x l) tensor stacking LR connectivity matrices of all training subjects
                                       n: the total number of subjects
                                       l: the dimensions of the LR connectivity matrices

                        subjects_ground_truth: (n × h x h) tensor stacking LR connectivity matrices of all training subjects
                                                n: the total number of subjects
                                                h: the dimensions of the LR connectivity matrices

                        args:          parsed command line arguments, to learn more about the arguments run: 
                                       python demo.py --help
                Output:
                        for each epoch, prints out the mean training MSE error


            
        test(model, test_adj,test_ground_truth,args)

                Inputs:
                        test_adj:      (n × l x l) tensor stacking LR connectivity matrices of all testing subjects
                                        n: the total number of subjects
                                        l: the dimensions of the LR connectivity matrices

                        test_ground_truth:      (n × h x h) tensor stacking LR connectivity matrices of all testing subjects
                                                 n: the total number of subjects
                                                 h: the dimensions of the LR connectivity matrices

                        see train method above for model and args.

                Outputs:
                        for each epoch, prints out the mean testing MSE error


    To evaluate our framework we used 5-fold cross-validation strategy.

    ---------------------------------------------------------------------
    Copyright 2020 Megi Isallari, Istanbul Technical University.
    All rights reserved.
    """


import torch
import numpy as np
import torch.optim as optim
from sklearn.model_selection import KFold
from preprocessing import *
from model import *
from train import *
import argparse



epochs = 200


parser = argparse.ArgumentParser(description='GSR-Net')
parser.add_argument('--epochs', type=int, default=epochs, metavar='no_epochs',
                help='number of episode to train ')
parser.add_argument('--lr', type=float, default=0.00005, metavar='lr',
                help='learning rate (default: 0.0001 using Adam Optimizer)')
parser.add_argument('--splits', type=int, default=3, metavar='n_splits',
                help='no of cross validation folds')
parser.add_argument('--lmbda', type=int, default=16, metavar='L',
                help='self-reconstruction error hyperparameter')
parser.add_argument('--lr_dim', type=int, default=LR_size, metavar='N',
                help='adjacency matrix input dimensions')
parser.add_argument('--hr_dim', type=int, default=HR_size, metavar='N',
                help='super-resolved adjacency matrix output dimensions')
parser.add_argument('--hidden_dim', type=int, default=268, metavar='N',
                help='hidden GraphConvolutional layer dimensions')
parser.add_argument('--padding', type=int, default=26, metavar='padding',
                help='dimensions of padding')
parser.add_argument('--embedding_size', type=int, default=32, metavar='embedding_size',
                help='node embedding size')
parser.add_argument('--early_stop_patient', type=int, default=10, metavar='early_stop_patient',
                help='early_stop_patience')



# Create an empty Namespace to hold the default arguments
args = parser.parse_args([]) 
print(args)

Namespace(epochs=200, lr=5e-05, splits=3, lmbda=16, lr_dim=160, hr_dim=268, hidden_dim=268, padding=26, embedding_size=32, early_stop_patient=10)


In [36]:
# SIMULATING THE DATA: EDIT TO ENTER YOUR OWN DATA
X = A_LR_train_matrix #np.random.normal(0, 0.5, (167, 160, 160))
Y = A_HR_train_matrix #np.random.normal(0, 0.5, (167, 288, 288))
print(X.shape)
print(Y.shape)

(167, 160, 160)
(167, 268, 268)


In [37]:
device = get_device()
print(device)

mps


In [38]:
def compute_degree_matrix_normalization_batch_numpy(adjacency_batch):
    """
    Optimizes the degree matrix normalization for a batch of adjacency matrices using NumPy.
    Computes the normalized adjacency matrix D^-1 * A for each graph in the batch.
    
    Parameters:
    - adjacency_batch: A NumPy array of shape (batch_size, num_nodes, num_nodes) representing
                       a batch of adjacency matrices.

    Returns:
    - A NumPy array of normalized adjacency matrices.
    """
    epsilon = 1e-6  # Small constant to avoid division by zero
    # Calculate the degree for each node in the batch
    d = adjacency_batch.sum(axis=2) + epsilon
    
    # Compute the inverse degree matrix D^-1 for the batch
    D_inv = np.reciprocal(d)[:, :, np.newaxis] * np.eye(adjacency_batch.shape[1])[np.newaxis, :, :]
    
    # Normalize the adjacency matrix using batch matrix multiplication
    normalized_adjacency_batch = np.matmul(D_inv, adjacency_batch)
    
    return normalized_adjacency_batch
X = compute_degree_matrix_normalization_batch_numpy(X)
A_LR_test_matrix = compute_degree_matrix_normalization_batch_numpy(A_LR_test_matrix)
print(X.shape)

(167, 160, 160)


In [39]:
cv = KFold(n_splits=args.splits, random_state=random_seed, shuffle=True)

ks = [0.9, 0.7, 0.6, 0.5]

best_model_fold_list = []
data_fold_list = []
i = 1
for train_index, test_index in cv.split(X):

    print(f"----- Fold {i} -----")


    netG = GSRNet(ks, args).to(device)
    optimizerG = optim.Adam(netG.parameters(), lr=args.lr)

    netD = Discriminator(input_dim=args.embedding_size, hidden_sizes=[16, 8]).to(device)
    optimizerD = optim.Adam(netD.parameters(), lr=args.lr)

    subjects_adj, test_adj, subjects_ground_truth, test_ground_truth = X[
        train_index], X[test_index], Y[train_index], Y[test_index]
    data_fold_list.append((subjects_adj, test_adj, subjects_ground_truth, test_ground_truth))


    ##################
    # subjects_adj = subjects_adj[:1]
    # subjects_ground_truth = subjects_ground_truth[:1]
    ##################

    # return_model = train_gan(
    #     netG, 
    #     optimizerG, 
    #     netD,
    #     optimizerD,
    #     subjects_adj, 
    #     subjects_ground_truth, 
    #     args, 
    #     test_adj=test_adj, 
    #     test_ground_truth=test_ground_truth
    # )

    return_model = train(netG, optimizerG, subjects_adj, subjects_ground_truth, args, test_adj, test_ground_truth)
    test_mae = test(return_model, test_adj, test_ground_truth, args)
    print(f"Val MAE: {test_mae}")
    best_model_fold_list.append(return_model)

    i += 1

    # break
    

----- Fold 1 -----
0.5 0.03 0.1
Epoch: 1, Train Loss: 0.315796, Train Error: 0.256734, Test Error: 0.243449
Epoch: 2, Train Loss: 0.297514, Train Error: 0.241573, Test Error: 0.214092
Epoch: 3, Train Loss: 0.253451, Train Error: 0.200415, Test Error: 0.175281
Epoch: 4, Train Loss: 0.230228, Train Error: 0.180214, Test Error: 0.167977
Epoch: 5, Train Loss: 0.225313, Train Error: 0.176938, Test Error: 0.166638
Epoch: 6, Train Loss: 0.221685, Train Error: 0.175916, Test Error: 0.166016
Epoch: 7, Train Loss: 0.219457, Train Error: 0.175613, Test Error: 0.165778
Epoch: 8, Train Loss: 0.217138, Train Error: 0.175156, Test Error: 0.165310
Epoch: 9, Train Loss: 0.216335, Train Error: 0.174871, Test Error: 0.165321
Epoch: 10, Train Loss: 0.214449, Train Error: 0.174674, Test Error: 0.164976
Epoch: 11, Train Loss: 0.214296, Train Error: 0.174390, Test Error: 0.164728
Epoch: 12, Train Loss: 0.211300, Train Error: 0.174088, Test Error: 0.164615
Epoch: 13, Train Loss: 0.210690, Train Error: 0.17373

In [40]:
from MatrixVectorizer import MatrixVectorizer

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.stats import pearsonr
from scipy.spatial.distance import jensenshannon
import torch
import networkx as nx

def evaluate(pred_matrices, gt_matrices):

    num_test_samples = gt_matrices.shape[0]

    # Initialize lists to store MAEs for each centrality measure
    mae_bc = []
    mae_ec = []
    mae_pc = []

    # # Iterate over each test sample
    # for i in range(num_test_samples):
    #     # Convert adjacency matrices to NetworkX graphs
    #     pred_graph = nx.from_numpy_array(pred_matrices[i], edge_attr="weight")
    #     gt_graph = nx.from_numpy_array(gt_matrices[i], edge_attr="weight")

    #     # Compute centrality measures
    #     pred_bc = nx.betweenness_centrality(pred_graph, weight="weight")
    #     pred_ec = nx.eigenvector_centrality(pred_graph, weight="weight")
    #     pred_pc = nx.pagerank(pred_graph, weight="weight")

    #     gt_bc = nx.betweenness_centrality(gt_graph, weight="weight")
    #     gt_ec = nx.eigenvector_centrality(gt_graph, weight="weight")
    #     gt_pc = nx.pagerank(gt_graph, weight="weight")

    #     # Convert centrality dictionaries to lists
    #     pred_bc_values = list(pred_bc.values())
    #     pred_ec_values = list(pred_ec.values())
    #     pred_pc_values = list(pred_pc.values())

    #     gt_bc_values = list(gt_bc.values())
    #     gt_ec_values = list(gt_ec.values())
    #     gt_pc_values = list(gt_pc.values())

    #     # Compute MAEs
    #     mae_bc.append(mean_absolute_error(pred_bc_values, gt_bc_values))
    #     mae_ec.append(mean_absolute_error(pred_ec_values, gt_ec_values))
    #     mae_pc.append(mean_absolute_error(pred_pc_values, gt_pc_values))

    # # Compute average MAEs
    # avg_mae_bc = sum(mae_bc) / len(mae_bc)
    # avg_mae_ec = sum(mae_ec) / len(mae_ec)
    # avg_mae_pc = sum(mae_pc) / len(mae_pc)

    # vectorize and flatten
    pred_1d = MatrixVectorizer.vectorize(pred_matrices).flatten()
    gt_1d = MatrixVectorizer.vectorize(gt_matrices).flatten()

    mae = mean_absolute_error(pred_1d, gt_1d)
    pcc = pearsonr(pred_1d, gt_1d)[0]
    js_dis = jensenshannon(pred_1d, gt_1d)

    print("MAE: ", mae)
    print("PCC: ", pcc)
    print("Jensen-Shannon Distance: ", js_dis)
    # print("Average MAE betweenness centrality:", avg_mae_bc)
    # print("Average MAE eigenvector centrality:", avg_mae_ec)
    # print("Average MAE PageRank centrality:", avg_mae_pc)
    # return mae, pcc, js_dis, avg_mae_bc, avg_mae_ec, avg_mae_pc



In [41]:
# for i in range(args.splits):
#     _, test_adjs, _, gt_matrices = data_fold_list[i]
#     model = best_model_fold_list[i]
#     model.eval()
#     pred_matrices = np.zeros(gt_matrices.shape)
#     with torch.no_grad():
#         for j, test_adj in enumerate(test_adjs):
#             pred_matrices[j] = model(torch.from_numpy(test_adj))[0].cpu()
#     evaluate(pred_matrices, gt_matrices)

MAE:  0.1341236565004594
PCC:  0.6254681190368102
Jensen-Shannon Distance:  0.28631966477106546
MAE:  0.14527797451449923
PCC:  0.5894614178362924
Jensen-Shannon Distance:  0.3004112047910723
MAE:  0.1451417916264916
PCC:  0.5836879948789552
Jensen-Shannon Distance:  0.2958431640728769


In [42]:
A_HR_train = pd.read_csv("../data/hr_train.csv")

pca = PCA(n_components=0.99, whiten=False)
A_HR_train_pca = pca.fit_transform(A_HR_train)
print(A_HR_train_pca.shape)

gm = GaussianMixture(n_components=5, random_state=random_seed)
A_HR_train_label = gm.fit_predict(A_HR_train_pca)
unique, counts = np.unique(A_HR_train_label, return_counts=True)
print(np.asarray((unique, counts)).T)

X = np.load('A_LR_train_matrix.npy')
y = np.load('A_HR_train_matrix.npy')

n_sample = X.shape[0]
X_train, X_val, y_train, y_val = train_test_split(
    X.reshape(n_sample, -1), 
    y.reshape(n_sample, -1), 
    test_size=0.20, 
    random_state=random_seed,
    stratify=A_HR_train_label
)

X_train = X_train.reshape(-1, LR_size, LR_size)
X_val = X_val.reshape(-1, LR_size, LR_size)
y_train = y_train.reshape(-1, HR_size, HR_size)
y_val = y_val.reshape(-1, HR_size, HR_size)

print("Train size:", len(X_train))
print("Val size:", len(X_val))

netG = GSRNet(ks, args).to(device)
optimizerG = optim.Adam(netG.parameters(), lr=args.lr)
args.early_stop_patient = 20
# print(args.early_stop_patient)
final_model = train(netG, optimizerG, X_train, y_train, args, X_val, y_val)

# final_model = best_model_fold_list[-1]

(167, 161)
[[ 0 13]
 [ 1 41]
 [ 2 14]
 [ 3 48]
 [ 4 51]]
Train size: 133
Val size: 34
0.5 0.03 0.1
Epoch: 1, Train Loss: 0.311588, Train Error: 0.252429, Test Error: 0.250990
Epoch: 2, Train Loss: 0.281891, Train Error: 0.225976, Test Error: 0.201362
Epoch: 3, Train Loss: 0.235306, Train Error: 0.183892, Test Error: 0.176163
Epoch: 4, Train Loss: 0.224539, Train Error: 0.175089, Test Error: 0.173045
Epoch: 5, Train Loss: 0.219150, Train Error: 0.173763, Test Error: 0.172302
Epoch: 6, Train Loss: 0.216750, Train Error: 0.173324, Test Error: 0.171891
Epoch: 7, Train Loss: 0.214777, Train Error: 0.173050, Test Error: 0.171370
Epoch: 8, Train Loss: 0.212875, Train Error: 0.172668, Test Error: 0.171106
Epoch: 9, Train Loss: 0.210912, Train Error: 0.172314, Test Error: 0.170619
Epoch: 10, Train Loss: 0.208983, Train Error: 0.172040, Test Error: 0.170457
Epoch: 11, Train Loss: 0.208138, Train Error: 0.171806, Test Error: 0.170268
Epoch: 12, Train Loss: 0.206285, Train Error: 0.171449, Test Er

In [43]:
final_model = best_model_fold_list[0]

In [44]:
print(args)

Namespace(epochs=200, lr=5e-05, splits=3, lmbda=16, lr_dim=160, hr_dim=268, hidden_dim=268, padding=26, embedding_size=32, early_stop_patient=20)


In [45]:
final_model.eval()
pred_train_matrices = np.zeros(y_train.shape)
pred_val_matrices = np.zeros(y_val.shape)
with torch.no_grad():
    for j, test_adj in enumerate(X_train):
        pred_train_matrices[j] = final_model(torch.from_numpy(test_adj))[0].cpu()

    print("Train")
    evaluate(pred_train_matrices, y_train)

    for j, test_adj in enumerate(X_val):
        pred_val_matrices[j] = final_model(torch.from_numpy(test_adj))[0].cpu()

    print("Val")
    evaluate(pred_val_matrices, y_val)

Train
MAE:  0.13350148288930702
PCC:  0.6067668252251874
Jensen-Shannon Distance:  0.29584277391954517
Val
MAE:  0.12906604591323537
PCC:  0.6478599592850003
Jensen-Shannon Distance:  0.2757063188368834


In [46]:
output_pred_list = []
final_model.eval()
with torch.no_grad():
    for i in range(A_LR_test_matrix.shape[0]):
        output_pred = final_model(torch.Tensor(A_LR_test_matrix[i]))[0].cpu()
        output_pred = MatrixVectorizer.vectorize(output_pred).tolist()
        output_pred_list.append(output_pred)

In [47]:
output_pred_stack = np.stack(output_pred_list, axis=0)
output_pred_1d = output_pred_stack.flatten()
assert output_pred_1d.shape == (4007136, )

In [48]:
df = pd.DataFrame({
    "ID": [i+1 for i in range(len(output_pred_1d))],
    "Predicted": output_pred_1d.tolist()
})

df

,ID,Predicted
0,1,0.576910
1,2,0.592497
2,3,0.641155
3,4,0.614392
4,5,0.565211
...,...,...
4007131,4007132,0.061797
4007132,4007133,0.011763
4007133,4007134,0.262607
4007134,4007135,0.129944


In [49]:
df.to_csv("gsr_gat_residual_node_edge_drop_50perecentdrop_fold0.csv", index=False)